# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,527.968201,0.676386,1234,528.644587,527.291815,1057.289174,1054.583630
6,495.660714,0.554186,1234,496.214900,495.106528,992.429801,990.213056
7,680.773231,0.266988,1234,681.040219,680.506244,1362.080439,1361.012487
8,547.900133,-0.072597,1234,547.827536,547.972730,1095.655073,1095.945460
9,475.769396,0.771884,1234,476.541279,474.997512,953.082559,949.995024
10,503.754832,-0.625453,1234,503.129378,504.380285,1006.258757,1008.760570
11,808.297651,-0.580565,1234,807.717086,808.878217,1615.434172,1617.756433
12,2.728219,0.040599,1234,2.768818,2.687620,5.537636,5.375241
13,195.840196,0.648793,1234,196.488989,195.191403,392.977978,390.382805
14,2.566992,-0.791312,1234,1.775680,3.358304,3.551359,6.716608


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-528.644587,527.291815
6,-496.214900,495.106528
7,-681.040219,680.506244
8,-547.827536,547.972730
9,-476.541279,474.997512
10,-503.129378,504.380285
11,-807.717086,808.878217
12,-2.768818,2.687620
13,-196.488989,195.191403
14,-1.775680,3.358304


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-528.644587,527.291815
6,-496.214900,495.106528
7,-681.040219,680.506244
8,-547.827536,547.972730
9,-476.541279,474.997512
10,-503.129378,504.380285
11,-807.717086,808.878217
12,-2.768818,2.687620
13,-196.488989,195.191403
14,-1.775680,3.358304


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-528.6445871 ,  527.29181499],
       [-496.21490026,  495.10652821],
       [-681.04021937,  680.50624354],
       [-547.82753628,  547.97272987],
       [-476.54127928,  474.99751205],
       [-503.12937833,  504.38028487],
       [-807.71708581,  808.87821656],
       [  -2.768818  ,    2.68762034],
       [-196.48898901,  195.19140269],
       [  -1.7756795 ,    3.35830423]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1055.289174,-1054.583630
6,990.429801,-990.213056
7,1360.080439,-1361.012487
8,1093.655073,-1095.945460
9,951.082559,-949.995024
10,1004.258757,-1008.760570
11,1613.434172,-1617.756433
12,3.537636,-5.375241
13,390.977978,-390.382805
14,1.551359,-6.716608


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
